In [1]:
import sys
from pathlib import Path

# Add the parent directory to the Python path
sys.path.append(str(Path().resolve().parent))

from spytial import *
from spytial.annotations import *
from spytial.annotations import flag


In [2]:
# Setup for performance metrics
import random
from time import sleep
perf_base = "spytial_perf"
def get_perf_path(structure, size):
    return perf_base + "_" + structure + "_" + f"{size}.json"
PI = 20
SIZES = [5, 10, 25, 50]

## Important Note

In CLRS, array-based stacks and queues depict empty array slots explicitly. To reproduce these figures in SPyTial, each position—full or empty—must correspond to an atom. Python’s None is a single global value, so using it would merge all empty positions into one indistinguishable atom. To preserve per-slot identity, we introduce a sentinel class (EmptySlot).

Without sentinels, a custom relationalizer would be required to emit distinct “empty” atoms for each index. The sentinel approach is simpler and keeps array arity visible to the layout engine, ensuring the figure matches the CLRS conventions.


For completeness, we include a version that uses `None` instead of sentinels at the end of the file, alongside corresponding visualizations (no empty slots).

In [3]:
class EmptySlot:
    def __repr__(self):
        return "<empty>"


# Stacks 


In [4]:

@attribute(field="top")
@attribute(field="capacity")
@inferredEdge(name="topElem", selector="{ a: ArrayStack,  x : object | (a.A)->(a.top)->x in idx}")
@orientation(selector='{x,y : (idx[object][object]) | @num:(x[idx[object]]) < @num:(y[idx[object]])}', directions=['directlyRight'])
@hideAtom(selector="ArrayStack.A + (int - idx[object][object])")
class ArrayStack:
    """
    CLRS-style stack implemented on top of a fixed-size array (0-based indexing).
    Uses A[0..capacity-1]; self.top is index of the top element (−1 means empty).
    Uses per-slot EmptySlot instances so None can be valid data and empty slots are visualizable.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        # use distinct EmptySlot instances for each slot
        self.A = [EmptySlot() for _ in range(capacity)]
        self.top = -1  # index of top element; -1 means empty

    def empty(self):
        return self.top == -1

    def size(self):
        return self.top + 1

    def push(self, x):
        if self.top + 1 == self.capacity:
            raise IndexError("Stack overflow")
        self.top += 1
        self.A[self.top] = x

    def pop(self):
        if self.top == -1:
            raise IndexError("Stack underflow")
        x = self.A[self.top]
        self.A[self.top] = EmptySlot()  # restore sentinel
        self.top -= 1
        return x

    def top_element(self):
        if self.top == -1:
            raise IndexError("Stack is empty")
        return self.A[self.top]

    def __repr__(self):
        return f"ArrayStack(size={self.size()}, capacity={self.capacity}, A={self.A})"
# ...existing code...

![d](img/clrs-stacks.png)

In [5]:
S = ArrayStack(7)

S.push(15)
S.push(6)
S.push(2)
S.push(9)

diagram(S)

S.push(17)
S.push(3)
diagram(S)

S.pop()
diagram(S)

# Queues



In [6]:

@attribute(field="head")
@attribute(field="capacity")
@attribute(field="tail")
@orientation(selector='{x,y : idx[object][object] | @num:(x[idx[object]]) < @num:(y[idx[object]])}', directions=['directlyRight'])
@hideAtom(selector="ArrayQueue.A + (int - idx[object][object])")
@inferredEdge(name="headElem", selector="{a : ArrayQueue, x : object | (a.A)->(a.head)->x in idx}")
@inferredEdge(name="tailElem", selector="{a : ArrayQueue, x : object | (a.A)->(a.tail)->x in idx}")

class ArrayQueue:
    """
    CLRS-style queue implemented on top of a fixed-size array (circular buffer),
    using head and tail indices. head points at the first element; tail points
    at the slot where the next element will be inserted. The array length is
    `capacity`; at most capacity-1 elements can be stored (one slot reserved).
    Uses per-slot EmptySlot instances as sentinels so None can be valid data.
    """
    def __init__(self, capacity):
        self.capacity = capacity
        # use distinct EmptySlot instances for each slot
        self.A = [EmptySlot() for _ in range(capacity)]
        self.head = 0
        self.tail = 0  # next free slot; queue empty when head == tail

    def empty(self):
        return self.head == self.tail

    def enqueue(self, x):
        # Full when advancing tail would hit head
        if (self.tail + 1) % self.capacity == self.head:
            raise IndexError("Queue overflow")
        self.A[self.tail] = x
        self.tail = (self.tail + 1) % self.capacity

    def dequeue(self):
        if self.head == self.tail:
            raise IndexError("Queue underflow")
        x = self.A[self.head]
        self.A[self.head] = EmptySlot()  # restore sentinel
        self.head = (self.head + 1) % self.capacity
        return x

    def front(self):
        if self.head == self.tail:
            raise IndexError("Queue is empty")
        return self.A[self.head]

    def __repr__(self):
        return f"ArrayQueue(head={self.head}, tail={self.tail}, capacity={self.capacity}, A={self.A})"


![qs](img/clrs-queues.png)

In [7]:
Q = ArrayQueue(12)

for v in range(11):
    Q.enqueue(v)


Q.dequeue()
Q.dequeue()
Q.dequeue()

Q.enqueue(-1)
Q.enqueue(-2)

diagram(Q)




# Without Sentinels

## Stacks

In [8]:
@attribute(field="top")
@attribute(field="capacity")
@inferredEdge(name="topElem", selector="{a, x : object |a in ArrayStackWNone and (a.A)->(a.top)->x in idx}")
@orientation(selector='{x,y : (idx[object][object] - NoneType) | @num:(x[idx[object]]) < @num:(y[idx[object]])}', directions=['directlyRight'])
@hideAtom(selector="ArrayStackWNone.A + (int - idx[object][object]) + NoneType")
class ArrayStackWNone:
    """
    CLRS-style stack implemented on top of a fixed-size array (0-based indexing).
    Uses A[0..capacity-1]; self.top is index of the top element (−1 means empty).
    """
    def __init__(self, capacity):
        self.capacity = capacity
        self.A = [None] * capacity  # 0-based array
        self.top = -1  # index of top element; -1 means empty

    def empty(self):
        return self.top == -1

    def size(self):
        return self.top + 1

    def push(self, x):
        if self.top + 1 == self.capacity:
            raise IndexError("Stack overflow")
        self.top += 1
        self.A[self.top] = x

    def pop(self):
        if self.top == -1:
            raise IndexError("Stack underflow")
        x = self.A[self.top]
        self.A[self.top] = None
        self.top -= 1
        return x

    def top_element(self):
        if self.top == -1:
            raise IndexError("Stack is empty")
        return self.A[self.top]

    def __repr__(self):
        return f"ArrayStack(size={self.size()}, capacity={self.capacity}, A={self.A})"


S = ArrayStackWNone(7)

S.push(15)
S.push(6)
S.push(2)
S.push(9)

diagram(S)

S.push(17)
S.push(3)
diagram(S)

S.pop()
diagram(S)

## Queues

In [9]:

@attribute(field="head")
@attribute(field="capacity")
@attribute(field="tail")
@orientation(selector='{x,y : idx[object][object] - NoneType | @num:(x[idx[object]]) < @num:(y[idx[object]])}', directions=['directlyRight'])
@hideAtom(selector="ArrayQueueWNone.A + NoneType + (int - idx[object][object])")
@inferredEdge(name="headElem", selector="{a, x : object | a in ArrayQueueWNone and (a.A)->(a.head)->x in idx}")
class ArrayQueueWNone:
    def __init__(self, capacity):
        self.capacity = capacity
        self.A = [None] * capacity   # None means empty
        self.head = 0
        self.tail = 0  # next free slot; empty when head == tail

    def empty(self):
        return self.head == self.tail

    def enqueue(self, x):
        if (self.tail + 1) % self.capacity == self.head:
            raise IndexError("Queue overflow")
        self.A[self.tail] = x
        self.tail = (self.tail + 1) % self.capacity

    def dequeue(self):
        if self.head == self.tail:
            raise IndexError("Queue underflow")
        x = self.A[self.head]
        self.A[self.head] = None   # mark slot empty
        self.head = (self.head + 1) % self.capacity
        return x

    def front(self):
        if self.head == self.tail:
            raise IndexError("Queue is empty")
        return self.A[self.head]

    def __repr__(self):
        return f"ArrayQueue(head={self.head}, tail={self.tail}, A={self.A})"
    


Q = ArrayQueueWNone(12)

for v in range(11):
    Q.enqueue(v)


Q.dequeue()
Q.dequeue()
Q.dequeue()

Q.enqueue(-1)
Q.enqueue(-2)

diagram(Q)




## Performance - Stack

In [10]:
STRUCTURE = "array_stack"
for size in SIZES:
    S = ArrayStack(size)
    values = random.sample(range(1, 1000), size)
    for val in values:
        S.push(val)
    
    print(f"{STRUCTURE}({size} elements): Rendering with perf_iterations={PI}...")
    diagram(S, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)
    sleep(2)

array_stack(5 elements): Rendering with perf_iterations=20...
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 28.83ms avg
  Render Layout: 68.06ms avg
  Total Time: 97.21ms avg
  Metrics saved to: spytial_perf_array_stack_5.json
array_stack(10 elements): Rendering with perf_iterations=20...
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 33.01ms avg
  Render Layout: 145.91ms avg
  Total Time: 179.26ms avg
  Metrics saved to: spytial_perf_array_stack_10.json
array_stack(25 elements): Rendering with perf_iterations=20...
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (20.4s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 53.65ms avg
  Render Layout: 494.86ms avg
  Total Time: 548.84ms avg
  Metrics saved to: spytial_perf_array_stack_25.j

## Performance - Queue

In [11]:
STRUCTURE = "array_queue"
for size in SIZES:
    Q = ArrayQueue(size + 1)  # Need capacity+1 to store 'size' elements (one slot reserved)
    values = random.sample(range(1, 1000), size)
    for val in values:
        Q.enqueue(val)
    
    print(f"{STRUCTURE}({size} elements): Rendering with perf_iterations={PI}...")
    diagram(Q, method="browser", perf_path=get_perf_path(STRUCTURE, size), perf_iterations=PI, headless=True)
    sleep(2)


array_queue(5 elements): Rendering with perf_iterations=20...
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.9s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 33.65ms avg
  Render Layout: 72.94ms avg
  Total Time: 107.00ms avg
  Metrics saved to: spytial_perf_array_queue_5.json
array_queue(10 elements): Rendering with perf_iterations=20...
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (10.0s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 34.96ms avg
  Render Layout: 147.62ms avg
  Total Time: 182.96ms avg
  Metrics saved to: spytial_perf_array_queue_10.json
array_queue(25 elements): Rendering with perf_iterations=20...
Running 20 iterations (timeout: 120s)...
  Progress: 20/20 iterations (20.2s elapsed)
✓ Headless benchmark completed: 20 iterations
  Generate Layout: 57.65ms avg
  Render Layout: 515.11ms avg
  Total Time: 573.14ms avg
  Metrics saved to: spytial_perf_array_queue_25.